# Numerical Integration of Functions

In science and engineering, many important quantities are obtained by
integrating functions.
Examples include the total energy radiated by a star (integral of
brightness over wavelength),
the probability of finding a quantum particle in a region (integral of
probability density), or
the synchrotron emissivity function (integral over electron
distribution function).

Analytical solutions are rare because real functions often come from
measurements, simulations, or complicated models.
In such cases, we rely on **numerical integration** (or *quadrature*)
to approximate
\begin{align}
  I = \int_a^b f(x)\,dx,
\end{align}
using only a finite number of function evaluations.

Numerical integration provides a controlled setting to study how
approximations are constructed, how accuracy depends on step size, and
how errors accumulate.
These lessons generalize directly to solving differential equations
and more complex simulations, making integration an essential starting
point for good numerical practice.

To study numerical integration systematically, it helps to begin with
a function whose integral we know exactly.
Consider $f(x) = e^x$.
Its definite integral from $a$ to $b$ is
\begin{align}
  I = \int_a^b e^x \, dx = e^b - e^a.
\end{align}
On the interval $[0,1]$, the exact value is $I = e - 1$.
This known result will allow us to check the accuracy of numerical
approximations as we vary step size and method.

Below is a simple plot of $f(x)=e^x$ on $[0,1]$ for visual reference.

In [ ]:
import numpy as np

def f(x):
    return np.exp(x)

# Define a fine grid for plotting
X = np.linspace(0, 1, 1025)
Y = f(X)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(X, Y, label=r"$f(x) = e^x$")
plt.fill_between(X, Y, alpha=1/3, label=r'$I = \int_a^b f(x) dx$')
plt.title(r'Function $f(x) = e^x$ on $[0,1]$')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()

## Riemann Sums

In undergraduate calculus, we learn the integral as the limit of
Riemann sums:
\begin{align}
  \int_a^b f(x) dx
  \equiv \lim_{n \to \infty} \sum_{i=1}^n f(x_i) \Delta x,
\end{align}
where $\Delta x \equiv (b-a)/n$.

In **numerical analysis**, however, we do **not** take the limit.
Instead, we keep the division of $[a, b]$ into $n$ subintervals and
use
\begin{align}
  I \approx \sum_{i=1}^n f(x_i)\,\Delta x,
\end{align}
as a practical approximation.
The choice of sampling point $x_i$ determines the accuracy.

Common choices for $x_i$ give us three variants:
* Left Riemann Sum: $x_i$ is the left endpoint.
* Right Riemann Sum: $x_i$ is the right endpoint.
* Midpoint (or middle) Riemann Sum: $x_i$ is the midpoint.

```{note} Analogy with Finite Differences

Just as we approximated derivatives using forward, backward, and
central differences,

Riemann sums mirror this structure:
* Left Riemann $\leftrightarrow$ forward difference
* Right Riemann $\leftrightarrow$ backward difference
* Midpoint Riemann $\leftrightarrow$ central difference

In derivatives, these approximations predict slopes.
In integrals, they approximate accumulated areas.
```

In [ ]:
n  = 8  # number of intervals
dx = 1 / n

Xl = np.linspace(0, 1-dx, n)
Xm = np.linspace(dx/2, 1-dx/2, n)
Xr = np.linspace(dx, 1, n)

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(12,4), sharey=True)

# Left Riemann Sum
for ax, Xs, name in zip(axes, [Xl, Xm, Xr], ['Left', 'Middle', 'Right']):
    Ys = f(Xs)
    ax.plot(X, Y)
    ax.bar(Xm, Ys, width=dx, align='center', color='C1', edgecolor=None, alpha=1/3)
    ax.scatter(Xs, Ys, color='C1', zorder=10)
    ax.set_title(f'{name} Riemann Sum')

In [ ]:
# HANDSON: There is another to define the middle Reimann sun,
#          where the sample points are "on vertex ", i.e.,
#          $x_i = a + (b-a) (i/n)$ for $i = 0, ..., n$.
#          What is the different between this definition and ours?
#          From the plots, what do you think about the relationships
#          between left, right, and this new middle Reimann sums?


### Computing Riemann Sums

Now that we have defined the left, right, and midpoint Riemann sums,
let us implement them and check their accuracy against the exact
result
\begin{align}
  I = \int_0^1 e^x \, dx = e - 1.
\end{align}

We begin with a simple case using $N=8$ subintervals.

In [ ]:
I = np.e - 1

print(f"         Exact value: {I :.8f}")

for ax, Xs, name in zip(axes, [Xl, Xm, Xr], ['Left', 'Middle', 'Right']):
    S = np.sum(f(Xs)) * dx  # multiple dx after sum
    print(f"{name:>8} Riemann Sum: {S:.8f}, error = {abs(I - S):.2e}")

In [ ]:
# HANDSON: try increasing $n$ and then observe the errors.


As the number of subintervals $n$ increases, all three Riemann sums
converge toward the exact value $e-1$.
However, the midpoint rule generally gives much better accuracy for
the same $n$, just as central differences are more accurate than
forward or backward differences.
This shows how the choice of sample points directly affects accuracy.

### Convergence of Riemann Sums

In numerical analysis, as we saw in earlier lecture, convergence means
studying how the error decreases as we refine the discretization.
For integration, we ask how does the error behave as we increase the
number of subintervals $n$?

To make comparisons easier, let's define a general function for
computing Riemann sums of any type:

In [ ]:
def RiemannSum(f, n=8, a=0, b=1, method='mid'):
    dx = (b-a)/n
    hx = dx/2
    
    if method.startswith('l'):
        X = np.linspace(a,    b-dx, n)  # left endpoints
    elif method.startswith('r'):
        X = np.linspace(a+dx, b,    n)  # right endpoints
    else:
        X = np.linspace(a+hx, b-hx, n)  # midpoints
    
    return np.sum(f(X)) * dx

Now we can study convergence systematically by increasing $n$.

In [ ]:
def convergence(f, N, I):
    # Compute absolute errors for each method
    El = [abs(RiemannSum(f, n, method='l') - I) for n in N]
    Em = [abs(RiemannSum(f, n, method='m') - I) for n in N]
    Er = [abs(RiemannSum(f, n, method='r') - I) for n in N]

    # Plot convergence behavior
    plt.loglog(N, El, 'o-',  label='Left Riemann Sum')
    plt.loglog(N, Em, '^--', label='Middle Riemann Sum')
    plt.loglog(N, Er, 's:',  label='Right Riemann Sum')

    # Reference slopes
    plt.loglog(N, 1.2e+0 / N,    ':', lw=1, label=r'$n^{-1}$')
    plt.loglog(N, 1.0e-1 / N**2, ':', lw=1, label=r'$n^{-2}$')

    plt.xlabel('Number of Subintervals $n$')
    plt.ylabel('Absolute Error')
    plt.title('Convergence of Riemann Sums')
    plt.legend()

In [ ]:
N = 2**np.arange(2,11)  # range of sample sizes
convergence(f, N, np.e-1)

From the plot, we observe:
* Left and Right Riemann sums converge at roughly a rate
  $\sim N^{-1}$.
* Midpoint Riemann sum converges faster, at a rate closer to
  $\sim N^{-2}$.

This higher order of accuracy explains why the midpoint method is much
more accurate for the same number of intervals.
The pattern mirrors what we saw with finite differences: forward and
backward methods are first-order accurate, while central methods are
second-order accurate.

### Testing Convergence with Different Functions

To determine if this trend holds generally, let's repeat the
convergence test with different functions:
a half-cycle of $\sin[(\pi/2)(1-x)]$ and
a quarter circle $\sqrt{1-x^2}$.

In [ ]:
f2 = lambda x: np.sin((np.pi/2) * (1-x))
f3 = lambda x: np.sqrt(1 - x*x)

plt.plot(X, f2(X))
plt.plot(X, f3(X))

In [ ]:
convergence(f2, N, 2/np.pi)

In [ ]:
convergence(f3, N, np.pi/4)

In [ ]:
# HANDSON: why does Middle Riemann Sum not converge as expected for
#          quarter circle?


Although the formal convergence rates hold in general, special
functions (e.g. highly oscillatory or discontinuous ones) may behave
differently.
Understanding these exceptions is a key part of numerical analysis.

As we progress, we will adopt the notation and framework used in
Numerical Recipes and related references.
These general-purpose methods extend the ideas of Riemann sums to more
accurate and flexible quadrature formulas, allowing us to tackle
realistic integration problems efficiently.

## Classical Formulas for Equally Spaced Abscissas

### Trapezoidal Rule

To move beyond Riemann sums, we now consider the trapezoidal rule, one
of the most widely used numerical integration formulas.

From here on, we adopt the vertex formulation:
\begin{align}
  x_i = a + i \Delta x,
  \quad \Delta x = \frac{b-a}{n},
  \quad i=0,1,\dots,n.
\end{align}

The trapezoidal rule approximates the area under a curve by replacing
each subinterval with a trapezoid.
For a single interval $[x_i, x_{i+1}]$ with width $h = x_{i+1} - x_i$,
we have:
\begin{align}
  \int_{x_i}^{x_{i+1}} f(x) dx
  \approx \frac{h}{2}\left[f(x_i) + f(x_{i+1})\right].
\end{align}

The error for one interval is of order $\mathcal{O}(h^3 f'')$, which
means the global error (after $n$ intervals) is $\mathcal{O}(h^2)$.
This makes the trapezoidal rule second-order accurate.
If $f(x)$ is linear (so $f''=0$), the approximation is exact.


In [ ]:
# Test function: quarter circle
Y = f(X)

# Plot with a coarse trapezoidal partition
n  = 4
Xt = np.linspace(0, 1, n+1)
Yt = f(Xt)

plt.plot(X, Y, label=r"$f(x) = \sqrt{1-x^2}$")
plt.scatter(Xt, Yt, color='C1')
plt.bar(Xt, Yt, width=Xt[1]-Xt[0], align='center',
        color='C1', edgecolor=None, alpha=1/3, label='Middle Riemann Sum')
plt.fill_between(Xt, Yt,
                 color='C2', alpha=1/3, label='Trapezoidal Rule')

plt.xlim(0, 1)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()

In [ ]:
# HANDSON: look at the figure carefully.
#          What is the result from the Middle Riemann Sum (on vertex)
#          compared to the trapezoidal rule?


In [ ]:
# HANDSON: reformulate the trapezoidal rule to speed it up.
